In [1]:
import numpy as np
from scipy.optimize import linprog

# Example cost matrix for three patients and two hospitals
# Costs include weighted components for wait times, load imbalance, and geographic proximity
cost_matrix = [
    [30, 20],  # Costs for Patient 1 to Hospital 1 and 2
    [25, 30],  # Costs for Patient 2 to Hospital 1 and 2
    [40, 35]   # Costs for Patient 3 to Hospital 1 and 2
]

# Flatten cost matrix for linear programming
c = [item for sublist in cost_matrix for item in sublist]

# Equality constraints: Each patient assigned to exactly one hospital
# 1 for hospitals corresponding to a patient, 0 otherwise
A_eq = [
    [1, 1, 0, 0, 0, 0],  # Patient 1
    [0, 0, 1, 1, 0, 0],  # Patient 2
    [0, 0, 0, 0, 1, 1]   # Patient 3
]
b_eq = [1, 1, 1]  # Each patient gets assigned to one hospital

# Inequality constraints: Hospitals should not exceed their capacity
A_ub = [
    [1, 0, 1, 0, 1, 0],  # Hospital 1
    [0, 1, 0, 1, 0, 1]   # Hospital 2
]
b_ub = [2, 2]  # Capacity of Hospital 1 and Hospital 2

# Bounds: Decision variables are binary (relaxed for LP, between 0 and 1)
x_bounds = [(0, 1)] * len(c)

# Solve the optimization problem using linear programming
result = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=x_bounds, method='highs')

# Reshape result to align with the patient-hospital structure
assignment_matrix = result.x.reshape(len(cost_matrix), len(cost_matrix[0]))

# Output the results
print("\n--- Optimal Assignment Results ---")
print("Assignment Matrix (Patient-Hospital):")
print(assignment_matrix)
print("Minimum Total Cost:", result.fun)

# Add weights for dynamic cost matrix
# For example, weights for wait time, load imbalance, and geographic proximity
alpha = 1  # Weight for predicted wait time
beta = 0.5  # Weight for load imbalance
gamma = 0.3  # Weight for geographic proximity

# Example weights and feature values
predicted_wait_times = np.array([[30, 20], [25, 30], [40, 35]])  # Predicted wait times (WTph)
load_imbalance = np.array([5, 10])  # Load imbalance for hospitals (Lh - L̄)
geographic_proximity = np.array([[10, 15], [12, 8], [20, 18]])  # Geographic proximity (GPph)

# Compute weighted costs for the matrix
cost_matrix_weighted = alpha * predicted_wait_times + beta * np.abs(load_imbalance - np.mean(load_imbalance)) + gamma * geographic_proximity

# Flatten weighted cost matrix for optimization
c_weighted = [item for sublist in cost_matrix_weighted for item in sublist]

# Re-run optimization with the weighted cost matrix
result_weighted = linprog(c_weighted, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=x_bounds, method='highs')

# Reshape result
assignment_matrix_weighted = result_weighted.x.reshape(len(cost_matrix_weighted), len(cost_matrix_weighted[0]))

# Output the results for the weighted model
print("\n--- Optimal Assignment Results (Weighted) ---")
print("Weighted Assignment Matrix (Patient-Hospital):")
print(assignment_matrix_weighted)
print("Minimum Total Weighted Cost:", result_weighted.fun)



--- Optimal Assignment Results ---
Assignment Matrix (Patient-Hospital):
[[-0.  1.]
 [ 1.  0.]
 [-0.  1.]]
Minimum Total Cost: 80.0

--- Optimal Assignment Results (Weighted) ---
Weighted Assignment Matrix (Patient-Hospital):
[[-0.  1.]
 [ 1.  0.]
 [-0.  1.]]
Minimum Total Weighted Cost: 97.25
